In [ ]:
#Import required packages
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from pandasql import sqldf
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, r2_score


In [ ]:
#Load data
medical_df=pd.read_csv('longyear-obese-hypertensive-40-57-medical-SMALL-sample.csv')

In [ ]:
#Drop duplicate rows
medical_df.drop_duplicates(inplace=True)

In [ ]:
#Check columns
medical_df.columns

In [ ]:
medical_df.shape #Check number of rows and columns

In [ ]:
medical_df['journey_id'].nunique()

In [ ]:
#Make a new column OH which indicates whether someone was diagnosed with obesity or hypertension that day
diag_columns = [col for col in medical_df.columns if col.startswith('diag_')]

# Initialize column with 0
medical_df['OH'] = 0

# Looping through each diag column 
for col in diag_columns:
    # Assign value to 'OH' according to diag columns. 1 if any diag column starts with 'E66' or 'I10', otherwise 0
    medical_df['OH'] = np.where((medical_df[col].str.startswith('E66')) | (medical_df[col].str.startswith('I10')) | (medical_df['OH'] == 1), 1, 0)

In [ ]:
#Make column d which indicates whether the patient was given ozempic
medical_df['d'] = np.where(medical_df['proc_code'] == 'J3490',1,0) #'J3490' is the proc_code for ozempic

In [ ]:
# Calculate counts for each year
counts = pd.to_datetime(medical_df[medical_df['d'] == 1]['claim_date']).dt.year.value_counts().sort_index()

# Plot 
fig, ax = plt.subplots(figsize=(14, 8))
ax = counts.plot(kind='bar', title="claim_date Volume Chart", ax=ax)
ax.set_xlabel("claim_date (Year)")
ax.set_ylabel("Frequency")

# Display percentage on the bars
total_entries = len(medical_df[medical_df['d'] == 1])
for i, val in enumerate(counts):
    ax.text(i, val + 0.5, f"{(val / total_entries):.2%}", ha='center')

plt.show()

Ozempic use has been increasing over the years. Data for 2023 is until September and adding next 3 months might/might not increase the count there. However, we can safely say that usage has been increasing every year.

In [ ]:
#Get first date of ozempic use
earliest_oz_date = medical_df[medical_df['d'] == 1]['claim_date'].min()
print("Earliest Ozempic date in dataset:", earliest_oz_date.date())

#Get first date in dataset
earliest_date = medical_df['claim_date'].min()
print("Earliest date in dataset:", earliest_date.date())

In [ ]:
specialty_count = medical_df[medical_df['d'] == 1]['hcp_specialty'].value_counts()

# Plotting
specialty_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of HCP Specialties')
plt.xlabel('Specialty')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Ozempic was prescribed most by emergency medicine doctors followed by internal medicine and nephrology.

In [ ]:
visit_type_count = medical_df[medical_df['d'] == 1]['visit_type'].value_counts()

# Plotting
visit_type_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of Visit Type')
plt.xlabel('Visit Type')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Ozempic is prescribed in highest numbers during outpatient visits. This can be a factor affecting treatment.

In [ ]:
age_count = medical_df[medical_df['d'] == 1]['patient_age'].value_counts().sort_index()

# Plotting
age_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of Patient Age')
plt.xlabel('Age')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Ozempic prescription increases with age but not for people who are above 55.

In [ ]:
#Datframe which has information for patients who had obesity/hypertension
oh_df = medical_df[medical_df['OH'] == 1]

# Histogram for Age distribution amongst people with obesity/hypertension
plt.figure(figsize=(6, 4))
plt.hist(oh_df['patient_age'], bins=10, edgecolor='black')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

Older people are diagnosed with obesity/hypertension more.

In [ ]:
state_count = medical_df[medical_df['d'] == 1]['patient_state'].value_counts()

# Plotting
state_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of Patient State')
plt.xlabel('State')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Majority of patients who are prescribed ozempic are from California.

In [ ]:
zip_count = medical_df[medical_df['d'] == 1]['patient_short_zip'].value_counts()

# Plotting
zip_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of Patient Zip Code')
plt.xlabel('Zip Code')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Majority chunk from California again as shown by state distribution. Within California, people from San Diego are prescribed ozempic the most. Ozempic is prescribed a lot in Southern California.

921.0 - San Diego

922.0 - Indio

908.0 - Long Beach

910.0 - Pasadena

911.0 - Pasadena

912.0 - Glendale, CA

853.0 - Glendale, AZ




In [ ]:

patient_gender_count = medical_df[medical_df['d'] == 1]['patient_gender'].value_counts()

# Plotting
patient_gender_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of Patient Gender')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.xticks(rotation=0, ha='right')
plt.tight_layout()
plt.show()

Females are prescribed more. They are diagnosed with obesity/hypertension more as shown in following graph.

In [ ]:
#Check distribution of gender in the dataset
plt.figure(figsize=(6, 4))
sns.countplot(x='patient_gender', data=oh_df)
plt.title('Distribution of Gender')
plt.show()

In [ ]:
place_of_service_count = medical_df[medical_df['d'] == 1]['place_of_service'].value_counts()

# Plotting
place_of_service_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of Place of Service')
plt.xlabel('Place of Place of Service')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()



Ozempic is prescribed considerbly higher in Offices as compared to other places

In [ ]:
#Payor distribution
payor_count = medical_df[medical_df['d'] == 1]['payor'].value_counts()

# Plotting
payor_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of payor')
plt.xlabel('Place of payor')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Ozempic is not covered by insurance for obesity treatment but is covered as diabetes treatment so, not going to use this in model

Amongst people diagnosed with obesity/hypertension, females form a greater percentage.

In [ ]:
# Visualize the correlation matrix
numeric_cols= oh_df.select_dtypes(include=[np.number]).columns.tolist()

plt.figure(figsize=(14, 10))
sns.heatmap(oh_df[numeric_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

No significant correlation discovered.

In [ ]:
#Keep only relevant columns
columns_to_keep = ['journey_id','claim_date','patient_state', 'patient_short_zip', 'patient_age', 'patient_gender', 'visit_type','hcp_specialty','place_of_service','diag_1', 'diag_2', 'diag_3', 'diag_4',
       'diag_5','proc_code','OH','d']
df = medical_df[columns_to_keep]
df.head()

In [ ]:
# Check for null values print only the columns with nulls in decending order of null coutns 
null_info = pd.DataFrame({'Column': df.isnull().sum().index, 'Null Count': df.isnull().sum().values}).sort_values(by='Null Count' , ascending=False)
null_info['Null Percentage'] = (null_info['Null Count'] / len(df)) * 100

# Display the result
print(null_info[null_info['Null Count'] > 0])

In [ ]:
#Drop rows with null values in the columns where null value percentage is low
df = df.dropna(subset = ['proc_code'])
df = df.dropna(subset = ['patient_age'])
df = df.dropna(subset = ['patient_gender'])
df = df.dropna(subset = ['patient_short_zip'])
df = df.dropna(subset = ['visit_type'])

#Add Unknown to where hcp_specialty and place_of_service are missing
df['hcp_specialty'] = df['hcp_specialty'].astype(str).fillna('UNKNOWN')
df['place_of_service'] = df['place_of_service'].astype(str).fillna('UNKNOWN')


diag_2, diag_3, diag_4, diag_5 columns can have NULL values when diag_1 has values so, no need to treat these. I am not treating diag_1 here as I want to check this column after grouping on jouney_id

In [ ]:
# Check for null values print only the columns with nulls in decending order of null coutns 
null_info = pd.DataFrame({'Column': df.isnull().sum().index, 'Null Count': df.isnull().sum().values}).sort_values(by='Null Count' , ascending=False)
null_info['Null Percentage'] = (null_info['Null Count'] / len(df)) * 100

# Display the result
print(null_info[null_info['Null Count'] > 0])

In [ ]:
#Make columns for dates of obesity/hypertension diagnosis date. This date will be later used to make outcome variable
df['OH_diagnosed_date'] = np.where(df['OH']==1,df['claim_date'],np.nan)
df['OH_not_diagnosed_date'] = np.where(df['OH']==0,df['claim_date'],np.nan)

In [ ]:
df.head()

In [ ]:
#Group the dataset 
grouping_code = """
SELECT journey_id, patient_state, patient_short_zip, patient_age, patient_gender, visit_type, hcp_specialty, place_of_service
, max(OH) as OH
, max(d) as d
,min(OH_diagnosed_date) as earliest_OH_diagnosed_date
,max(OH_not_diagnosed_date) as last_OH_not_diagnosed_date
FROM df
GROUP BY 1,2,3,4,5,6,7,8
"""
grouped_df = sqldf(grouping_code)


In [ ]:
# We are interested in people who were diagnosed with obesity/hypertension at some point and were 40 and 75 years of age 
filtered_df = grouped_df[(grouped_df['OH']==1)& (grouped_df['patient_age']>40) & (grouped_df['patient_age']<75)]


In [ ]:
# Make outcome variable 'y' which indicates whether the patient got treated successfully or not - if the patient was diagnosed with obesity'hypertension at some point and was not diagnosed with them later, we can say that the patient was treated
filtered_df['y'] = np.where(filtered_df['last_OH_not_diagnosed_date'] > filtered_df['earliest_OH_diagnosed_date'], 1, 0)

In [ ]:
filtered_df.head()

In [ ]:
##Perform one hot encoding on gender
one_hot_df = pd.get_dummies(filtered_df, columns=['patient_gender'],drop_first=True,dtype=float)

#Perform label encoding on remaining categorical variables
# Columns to label encode
columns_to_encode = ['patient_state','patient_short_zip','patient_age','visit_type','hcp_specialty','place_of_service']
                                                       

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each column
for column in columns_to_encode:
    one_hot_df[column] = label_encoder.fit_transform(one_hot_df[column])

In [ ]:
one_hot_df.head()

Causal Analysis Setup:
Potential endogeneity issues can arise while determining effect of ozempic on the target population. Whether a patient was given ozempic or not is not completely random, therefore trying to look at patients who got ozempic and who did not and directly calculating treatment effect would be misleading. We need to control confounding factors here, which means there are factors influencing whether a patient was given ozempic or not which need to be controlled. For example, a patient might have been givn ozempic in hospital but not during a home visit.

We need to measure effect of part od d which is independent of x. For that, we will estimate d from x and use that predicted d as well while running regressing on y. This predicted d will capture the endogeneity effect and we will get an estimate of treatment effect.

I included Patient state, Patient Zip Code, Patient Age, Patient Gender, Visit Type, HCP Specialty, Place of Service in the model. Looking at the distribution of people who got ozempic across these, there was difference in values of these variables. Therefore, I included them. Also, 

Visit Type/Place of Service - Ozempic might have been prescribed more in office settings than home because in hospitals, the patient might have been directed to the doctor who would prescribe ozempic.

HCP Specialty - Internal medicine doctors prescribe more ozempic than others

Demographic factors could have been a consideration for prescription

In [ ]:
#Keep columns that are to be used in model
columns_to_drop = ['journey_id','OH','earliest_OH_diagnosed_date','last_OH_not_diagnosed_date']
filtered_df2 = one_hot_df.drop(columns = columns_to_drop)

In [ ]:
#Split the data into testing and training data
train, test = train_test_split(filtered_df2, train_size=0.8, random_state=0)

In [ ]:
#Pull x, d and y separately
x_train= train.drop(['d','y'],axis=1)
x_test= test.drop(['d','y'],axis=1)
y_train= train['y']
y_test= test['y']
d_train= train['d']
d_test= test['d']

First lasso with d~x

In [ ]:
#Find best alpha through cross-validation

from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(alphas=None, cv=5, random_state=0)

# Fit the model
lasso_cv.fit(x_train, d_train)

# Best alpha
print(f"Best alpha: {lasso_cv.alpha_}")

In [ ]:
#Fit lasso and predict treatment variable and store it in another variable 
lasso = Lasso(alpha=lasso_cv.alpha_)
lasso.fit(x_train, d_train)
d_train_pred = lasso.predict(x_train)
d_test_pred = lasso.predict(x_test)

In [ ]:
x_train.head()

In [ ]:
print("Coefficients for first lasso:", lasso.coef_)

Lasso reduced factors - patient state, patient age, Visit Type and Patient Gender
Whether a patient gets treated with ozempic or not depends on which zip code the person belongs to, Specialty of doctor and place of service

Second lasso (partial) with y~d, predicted d, x

In [ ]:
#Make new x which has d and predicted d as well
x_train2 = train.drop('y',axis=1)
x_train2['d_predicted'] = d_train_pred
x_test2 = test.drop('y',axis=1)
x_test2['d_predicted'] = d_test_pred

In [ ]:
x_train2.head()

In [ ]:
x_train2.shape

In [ ]:
#Function for partial lasso
import numpy as np
from scipy.optimize import minimize
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import log_loss

class SelectiveRegularizationLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, penalty_indices, penalty_weights, tol=1e-4, max_iter=100):
        self.penalty_indices = penalty_indices
        self.penalty_weights = penalty_weights
        self.tol = tol
        self.max_iter = max_iter
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def _loss(self, coef, X, y):
        predictions = self._sigmoid(X @ coef)
        # Basic log loss
        basic_loss = log_loss(y, predictions)
        
        # Regularization term, selectively applied with L1 penalty
        reg_term = sum(self.penalty_weights[i] * abs(coef[self.penalty_indices[i]]) for i in range(len(self.penalty_indices)))
        
        return basic_loss + reg_term
    
    def fit(self, X, y):
        # Add intercept term
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        
        initial_coef = np.zeros(X.shape[1])
        result = minimize(self._loss, initial_coef, args=(X, y), method='SLSQP', tol=self.tol, options={'maxiter': self.max_iter})
        
        self.coef_ = result.x
        return self
    
    def predict_proba(self, X):
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        proba = self._sigmoid(X @ self.coef_)
        return np.vstack([1-proba, proba]).T
    
    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

In [ ]:
lasso_cv = LassoCV(alphas=None, cv=5, random_state=0)

# Fit the model
lasso_cv.fit(x_train2, y_train)

# Best alpha
print(f"Best alpha: {lasso_cv.alpha_}")

In [ ]:
x_train2 = x_train2.values
y_train2 = y_train.values
x_test2 = x_test2.values
y_test2 = y_test.values

In [ ]:
penalty_indices = [1,2,3,4,5,6,8] # Apply penalties to all coefficients except that of d and d_predicted
penalty_weights = [0.0014,0.0014,0.0014,0.0014,0.0014,0.0014,0.0014] # Penalty weights for these coefficients
# penalty_weights = [0.1,0.1,0.1,0.1,0.1,0.1,0.1] # Penalty weights for these coefficients
# penalty_weights = [0.5,0.5,0.5,0.5,0.5,0.5,0.5] # Penalty weights for these coefficients

model = SelectiveRegularizationLogisticRegression(penalty_indices=penalty_indices, penalty_weights=penalty_weights)
model.fit(x_train2, y_train)

In [ ]:
print("Coefficients for features:", model.coef_)

In [ ]:
#Predict and evaluate model
y_pred = model.predict(x_test2)
mse = mean_squared_error(y_test2, y_pred)
rmse = np.sqrt(mse)
r2_score = r2_score(y_test2, y_pred)
print("MSE:", mse)
print("RMSE:", rmse)
print("R^2:", r2_score)

R^2 is negative here which states that null model would be better in prediction of outcome variable. So, the model is not effective at predicting whether a person gets cured. This might be because small data is used and it has only a few patients who were prescribed ozempic and then got treated of obesity/hypertension. Taking large data might make the performance of the model better.

Therefore, we can rely on the coefficient of d here, which is positive. If the model was good, and the coefficient was d was positive, we could have concluded that ozempic helps in treating obesity/hypertension. But we can not conclude in this case.

In [ ]:
oz_count = filtered_df2[(filtered_df2['d'] == 1)].shape[0]
print("In the dataset, number of patients who were given ozempic are", oz_count)
print("And total number of patients are ",filtered_df2.shape[0])

In [ ]:
zip_count = medical_df[medical_df['d'] == 1]['patient_short_zip'].value_counts()

# Plotting
zip_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of Patient (who were treated with ozempic) Zip Code')
plt.xlabel('Zip Code')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

Majority chunk from California again as shown by state distribution. Within California, people from San Diego are prescribed ozempic the most. Ozempic is prescribed a lot in Southern California.

921.0 - San Diego

922.0 - Indio

908.0 - Long Beach

910.0 - Pasadena

911.0 - Pasadena

912.0 - Glendale, CA

853.0 - Glendale, AZ

In [ ]:
zip_count = medical_df[medical_df['OH'] == 1]['patient_short_zip'].value_counts()

# Plotting
zip_count.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Count of Patient (who has obesity/hypertension) Zip Code')
plt.xlabel('Zip Code')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

900 - LA does not show up in patients who got ozempic however, LA has a lot of patient who were diagnosed with obesity/hypertension

In [ ]:
data = {
    'City': ['San Diego', 'Indio','Long Beach','Glendale'],
    'Population': ['1.39M', '88,542','147,321','195,477'],
    'Median Age': [35.4, 40.7, '35.1','41.3'],
    'Poverty Rate': ['11.6%', '14.5%','15.9%','13.9%'],
    'Median Household Income': ['$89,457', '$59,399','$66,041','$74,644'],
    'Median Property Value': ['$664,000', '$311,700','$520,200','$866,900'],
    'Employed Population': [696506, np.nan,np.nan,85382]
}

cities_df = pd.DataFrame(data)
cities_df

Information related to zip codes does not make the impacting factors clear.